## A01 - Simple usage of pretrained SatCLIP embeddings

To obtained pretrained **SatCLIP** embeddings, first install the repository.

In [1]:
# current directory
!pwd

/mnt/pan/SOM_CVRI_JXD101/DATA_SCIENCE/GIS/Multi_Geo_CLIP/satclip/notebooks


In [ ]:
!rm -r sample_data .config # Empty current directory
!git clone https://github.com/microsoft/satclip.git . # Clone SatCLIP repository

Cloning into '.'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 131 (delta 14), reused 16 (delta 4), pack-reused 95
Receiving objects: 100% (131/131), 9.01 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (48/48), done.


Now install the required Python packages.

In [ ]:
!pip install lightning --quiet
!pip install rasterio --quiet
!pip install torchgeo --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.7/670.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

You can find a list of pretrained SatCLIP models [here](https://github.com/microsoft/satclip#pretrained-models). Let's download a SatCLIP using a ResNet18 vision encoder and $L=10$ Legendre polynomials for spherical harmonics calculation in the location encoder.

In [ ]:
!wget 'https://satclip.z13.web.core.windows.net/satclip/satclip-resnet18-l10.ckpt'

--2023-12-07 14:55:28--  https://satclip.z13.web.core.windows.net/satclip/satclip-resnet18-l10.ckpt
Resolving satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)... 52.239.221.231
Connecting to satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)|52.239.221.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57201927 (55M) [application/zip]
Saving to: ‘satclip-resnet18-l10.ckpt’

satclip-resnet18-l1 100%[===================>]  54.55M  91.7MB/s    in 0.6s    

2023-12-07 14:55:29 (91.7 MB/s) - ‘satclip-resnet18-l10.ckpt’ saved [57201927/57201927]



Load required packages.

In [ ]:
import sys
sys.path.append('./satclip')

import torch
from load import get_satclip

And now we can sample some random lat/lon locations for which we obtain SatCLIP embeddings.

In [ ]:
satclip_path = 'satclip-resnet18-l10.ckpt'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c = torch.randn(32, 2) # Represents a batch of 32 locations (lon/lat)

model = get_satclip(satclip_path, device=device) # Only loads location encoder by default
model.eval()
with torch.no_grad():
  emb  = model(c.double().to(device)).detach().cpu()

using pretrained moco resnet18


Downloading: "https://huggingface.co/torchgeo/resnet18_sentinel2_all_moco/resolve/main/resnet18_sentinel2_all_moco-59bfdff9.pth" to /root/.cache/torch/hub/checkpoints/resnet18_sentinel2_all_moco-59bfdff9.pth
100%|██████████| 42.8M/42.8M [00:00<00:00, 59.4MB/s]


In [ ]:
emb.shape

torch.Size([32, 256])